<a href="https://colab.research.google.com/github/ArulSamuel3/ArulSamuel3.github.io/blob/main/Cuda_sample_using_blocks_threads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4q6p8muy
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4q6p8muy
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [91]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


**Addition of two values using CUDA** ✅

In [92]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values  
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

result is 8





---



---


Squaring arraay elements only using Threads......

---



---



In [4]:
%%cu
#include <iostream>
#include <chrono>
typedef std::chrono::high_resolution_clock Clock;
typedef std::chrono::time_point<Clock> TimePoint;

__global__ void square(float * d_out,float * d_in)
{
    int idx = threadIdx.x;
    float f = d_in[idx];
    d_out[idx]=f+f;
}

int main(int argc , char ** argv)
{
    const int ARRAY_SIZE=6;
    const int ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    
    float h_in[ARRAY_SIZE];
    for(int i=0; i< ARRAY_SIZE;i++)
    {
        h_in[i]=float(i);
    }
    float h_out[ARRAY_SIZE];
float * d_in;
float * d_out;

cudaMalloc((void **)&d_in,ARRAY_BYTES);
cudaMalloc((void **)&d_out,ARRAY_BYTES);
 
TimePoint timer1 = Clock::now();

cudaMemcpy(d_in,h_in,ARRAY_BYTES,cudaMemcpyHostToDevice);

square<<<1,ARRAY_SIZE>>>(d_out,d_in);

cudaMemcpy(h_out,d_out,ARRAY_BYTES,cudaMemcpyDeviceToHost);
 
std::chrono::duration<double> secs = (Clock::now() - timer1);
double duration = 1e6 * secs.count();

std::cout << " took " << duration << " microseconds to complete." << std::endl;

for(int i=0;i<ARRAY_SIZE;i++)
{
   // printf("%f",h_out[i]);
   // printf(((i+4)!=3)?"\t":"\n");
    std::cout << h_out[i];
    std::cout << (((i+4) != 3) ? "\t" : "\n");
}



cudaFree(d_in);
cudaFree(d_out);
 return 0;
}

 took 68.905 microseconds to complete.
0	2	4	6	8	10	


same square function contaning both normal and Cuda and corresponding unit test as well!!!

In [5]:
%%cu
#include <iostream>
#include <chrono>
typedef std::chrono::high_resolution_clock Clock;
typedef std::chrono::time_point<Clock> TimePoint;

__global__ void square(float * d_out,float * d_in)
{
    int idx = threadIdx.x;
    float f = d_in[idx];
    d_out[idx]=f+f;
}

void normal_add(float *in,float * out,const int ARRAY_SIZE)
{
      
    for(int i =0;i<ARRAY_SIZE;i++){
        out[i]=in[i]+in[i];
    }
}



int main(int argc , char ** argv)
{
    const int ARRAY_SIZE=256;
    const int ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    
    float h_in[ARRAY_SIZE];
    for(int i=0; i< ARRAY_SIZE;i++)
    {
        h_in[i]=float(i);
    }
    float h_out[ARRAY_SIZE];
    float h_out_C[ARRAY_SIZE];
 
TimePoint timer = Clock::now();
normal_add(h_in,h_out_C,ARRAY_SIZE);
std::chrono::duration<double> sec = (Clock::now() - timer);
double time = 1e6 * sec.count();

std::cout << "normal program takes " << time << " microseconds to complete." << std::endl;
 
float * d_in;
float * d_out;

cudaMalloc((void **)&d_in,ARRAY_BYTES);
cudaMalloc((void **)&d_out,ARRAY_BYTES);
 
TimePoint timer1 = Clock::now();

cudaMemcpy(d_in,h_in,ARRAY_BYTES,cudaMemcpyHostToDevice);


square<<<1,ARRAY_SIZE>>>(d_out,d_in);

cudaMemcpy(h_out,d_out,ARRAY_BYTES,cudaMemcpyDeviceToHost);
 
std::chrono::duration<double> secs = (Clock::now() - timer1);
double duration = 1e6 * secs.count();

std::cout << "CUDA program takes " << duration << " microseconds to complete." << std::endl;

/*for(int i=0;i<ARRAY_SIZE;i++)
{
    std::cout << h_out[i];
    std::cout << (((i+4) != 3) ? "\t" : "\n");
}

for(int i=0;i<ARRAY_SIZE;i++)
{
    std::cout << h_out_C[i];
    std::cout << (((i+4) != 3) ? "\t" : "\n");
}*/


cudaFree(d_in);
cudaFree(d_out);
return 0;
}

normal program takes 0.718 microseconds to complete.
CUDA program takes 58.376 microseconds to complete.



In [6]:
%%cu
#include <iostream>
#include <cuda_runtime.h>

int main()
{
    int maxThreadsPerBlock;
    cudaDeviceGetAttribute(&maxThreadsPerBlock, cudaDevAttrMaxThreadsPerBlock, 0);

    std::cout << "Maximum threads per block: " << maxThreadsPerBlock ;

    return 0;
}


Maximum threads per block: 1024




---


**squaring the array elements using threads and blocks..!!!!**


---



In [7]:
%%cu

#include <iostream>
#include <chrono>
#include <assert.h>

typedef std::chrono::high_resolution_clock Clock;
typedef std::chrono::time_point<Clock> TimePoint;

__global__ void square(float* d_out, float* d_in)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    float f = d_in[idx];
    d_out[idx] = f * f;
}

void normal_add(float* in, float* out, const int ARRAY_SIZE)
{
    for (int i = 0; i < ARRAY_SIZE; i++) {
        out[i] = in[i] * in[i];
    }
}

void unit_test(float *out1,float* out2, const int ARRAY_SIZE)
{
    for(int i=0;i<ARRAY_SIZE ; i++)
    {
        assert(out1[i]==out2[i]);
    }
    std :: cout << "ALL TESTS PASSED\n";
}


int main(int argc, char** argv)
{
    const int ARRAY_SIZE = 100000;
 
    const int ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 
    const int THREADS_PER_BLOCK = 1024;  
 
    int numBlocks = (ARRAY_SIZE + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
 
    float h_in[ARRAY_SIZE];
    for (int i = 0; i < ARRAY_SIZE; i++) 
    {
        h_in[i] = float(i);
    }
 
    float h_out[ARRAY_SIZE];
    float h_out_C[ARRAY_SIZE];

    TimePoint timer = Clock::now();
 
    normal_add(h_in, h_out_C, ARRAY_SIZE);
 
    std::chrono::duration<double> sec = (Clock::now() - timer);
    double time = 1e6 * sec.count();
    std::cout << "Normal program takes " << time << " microseconds to complete." << std::endl;

    float* d_in;
    float* d_out;

    cudaMalloc((void**)&d_in, ARRAY_BYTES);
    cudaMalloc((void**)&d_out, ARRAY_BYTES);

    cudaMemcpy(d_in, h_in, ARRAY_BYTES, cudaMemcpyHostToDevice);
 
    TimePoint timer1 = Clock::now();

    square<<<numBlocks, THREADS_PER_BLOCK>>>(d_out, d_in);

    std::chrono::duration<double> secs = (Clock::now() - timer1);
    double duration = 1e6 * secs.count();
    std::cout << "CUDA program takes " << duration << " microseconds to complete." << std::endl;

    cudaMemcpy(h_out, d_out, ARRAY_BYTES, cudaMemcpyDeviceToHost);


    unit_test(h_out,h_out_C,ARRAY_SIZE);

    cudaFree(d_in);
    cudaFree(d_out);

    return 0;
}


Normal program takes 380.597 microseconds to complete.
CUDA program takes 18.216 microseconds to complete.
ALL TESTS PASSED



**Square function for 2D arrays.....!!!!!!**



In [8]:
%%cu

#include <iostream>
#include <chrono>
#include <assert.h>

typedef std::chrono::high_resolution_clock Clock;
typedef std::chrono::time_point<Clock> TimePoint;

__global__ void square(float* d_out, float* d_in, int rows, int cols)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int idx = row * cols + col;
    if (row < rows && col < cols) {
        float f = d_in[idx];
        d_out[idx] = f * f;
    }
}

void normal_add(float* in, float* out, int rows, int cols)
{
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            int idx = i * cols + j;
            out[idx] = in[idx] * in[idx];
        }
    }
}

void unit_test(float *out1, float* out2, int rows, int cols)
{
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            int idx = i * cols + j;
            assert(out1[idx] == out2[idx]);
        }
    }
    std::cout << "ALL TESTS PASSED\n";
}

int main(int argc, char** argv)
{
    const int ROWS = 10;
    const int COLS = 10;
    const int ARRAY_SIZE = ROWS * COLS;

    const int THREADS_PER_BLOCK_X = 16;
    const int THREADS_PER_BLOCK_Y = 16;
    const int BLOCKS_X = (COLS + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X;
    const int BLOCKS_Y = (ROWS + THREADS_PER_BLOCK_Y - 1) / THREADS_PER_BLOCK_Y;

    float h_in[ARRAY_SIZE];
    for (int i = 0; i < ROWS; i++) {
        for (int j = 0; j < COLS; j++) {
            int idx = i * COLS + j;
            h_in[idx] = float(idx);
        }
    }

    float h_out[ARRAY_SIZE];
    float h_out_C[ARRAY_SIZE];

    TimePoint timer = Clock::now();
    normal_add(h_in, h_out_C, ROWS, COLS);
    std::chrono::duration<double> sec = (Clock::now() - timer);
    double time = 1e6 * sec.count();
    std::cout << "Normal program takes " << time << " microseconds to complete." << std::endl;

    float* d_in;
    float* d_out;
    cudaMalloc((void**)&d_in, ARRAY_SIZE * sizeof(float));
    cudaMalloc((void**)&d_out, ARRAY_SIZE * sizeof(float));

    cudaMemcpy(d_in, h_in, ARRAY_SIZE * sizeof(float), cudaMemcpyHostToDevice);

    dim3 blockSize(THREADS_PER_BLOCK_X, THREADS_PER_BLOCK_Y);
    dim3 gridSize(BLOCKS_X, BLOCKS_Y);

    TimePoint timer1 = Clock::now();
    square<<<gridSize, blockSize>>>(d_out, d_in, ROWS, COLS);
    std::chrono::duration<double> secs = (Clock::now() - timer1);
    double duration = 1e6 * secs.count();
    std::cout << "CUDA program takes " << duration << " microseconds to complete." << std::endl;

    cudaMemcpy(h_out, d_out, ARRAY_SIZE * sizeof(float), cudaMemcpyDeviceToHost);

    unit_test(h_out, h_out_C, ROWS, COLS);

    cudaFree(d_in);
    cudaFree(d_out);
 
    return 0;
}


Normal program takes 0.538 microseconds to complete.
CUDA program takes 17.162 microseconds to complete.
ALL TESTS PASSED



### **Matrix Addition of 2d array.....!!!!!**

In [24]:
%%cu

#include <iostream>
#include <chrono>
#include <assert.h>

typedef std::chrono::high_resolution_clock Clock;
typedef std::chrono::time_point<Clock> TimePoint;

__global__ void matrixAdd(float* d_out, float* d_in1, float* d_in2, int rows, int cols)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int idx = row * cols + col;
    if (row < rows && col < cols) {
        d_out[idx] = d_in1[idx] + d_in2[idx];
    }
}

void normalMatrixAdd(float* in1, float* in2, float* out, int rows, int cols)
{
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            int idx = i * cols + j;
            out[idx] = in1[idx] + in2[idx];
        }
    }
}

void unitTestMatrixAdd(float* out1, float* out2, int rows, int cols)
{
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            int idx = i * cols + j;
            assert(out1[idx] == out2[idx]);
        }
    }
    std::cout << "ALL TESTS PASSED\n";
}

int main(int argc, char** argv)
{
    const int ROWS = 5;
    const int COLS = 5;
    const int MATRIX_SIZE = ROWS * COLS;

    const int THREADS_PER_BLOCK_X = 16;
    const int THREADS_PER_BLOCK_Y = 16;
    const int BLOCKS_X = (COLS + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X;
    const int BLOCKS_Y = (ROWS + THREADS_PER_BLOCK_Y - 1) / THREADS_PER_BLOCK_Y;

    float h_in1[MATRIX_SIZE];
    float h_in2[MATRIX_SIZE];
 
    for (int i = 0; i < ROWS; i++) {
        for (int j = 0; j < COLS; j++) {
            int idx = i * COLS + j;
            h_in1[idx] = float(idx);
            h_in2[idx] = float(idx * 2);
        }
    }

    float h_out[MATRIX_SIZE];
    float h_out_C[MATRIX_SIZE];

    TimePoint timer = Clock::now();
    normalMatrixAdd(h_in1, h_in2, h_out_C, ROWS, COLS);
    std::chrono::duration<double> sec = (Clock::now() - timer);
    double time = 1e6 * sec.count();
    std::cout << "Normal program takes " << time << " microseconds to complete." << std::endl;

    float* d_in1;
    float* d_in2;
    float* d_out;
    cudaMalloc((void**)&d_in1, MATRIX_SIZE * sizeof(float));
    cudaMalloc((void**)&d_in2, MATRIX_SIZE * sizeof(float));
    cudaMalloc((void**)&d_out, MATRIX_SIZE * sizeof(float));

    cudaMemcpy(d_in1, h_in1, MATRIX_SIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_in2, h_in2, MATRIX_SIZE * sizeof(float), cudaMemcpyHostToDevice);

    dim3 blockSize(THREADS_PER_BLOCK_X, THREADS_PER_BLOCK_Y);
    dim3 gridSize(BLOCKS_X, BLOCKS_Y);

    TimePoint timer1 = Clock::now();
    matrixAdd<<<gridSize, blockSize>>>(d_out, d_in1, d_in2, ROWS, COLS);
    std::chrono::duration<double> secs = (Clock::now() - timer1);
    double duration = 1e6 * secs.count();
    std::cout << "CUDA program takes " << duration << " microseconds to complete." << std::endl;

    cudaMemcpy(h_out, d_out, MATRIX_SIZE * sizeof(float), cudaMemcpyDeviceToHost);

    unitTestMatrixAdd(h_out, h_out_C, ROWS, COLS);
 
 

    cudaFree(d_in1);
    cudaFree(d_in2);
    cudaFree(d_out);

    return 0;
}


Normal program takes 1782.89 microseconds to complete.
CUDA program takes 24.106 microseconds to complete.
ALL TESTS PASSED



### ***Shared Memory example....!!!***

---


Shared memory is a special type of memory available on NVIDIA GPUs that is 
shared among threads within a thread block. It is physically located on the GPU chip and is much faster to access compared to global memory, which is located off-chip and shared among all threads.

In [99]:
%%cu

#include <iostream>
#include <chrono>
#include <assert.h>

typedef std::chrono::high_resolution_clock Clock;
typedef std::chrono::time_point<Clock> TimePoint;

const int BLOCK_SIZE = 32;

__global__ void matrixAdd(float* d_out, float* d_in1, float* d_in2, int rows, int cols)
{
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int bx = blockIdx.x;
    int by = blockIdx.y;

    int row = by * blockDim.y + ty;
    int col = bx * blockDim.x + tx;
    int idx = row * cols + col;

    __shared__ float s_in1[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ float s_in2[BLOCK_SIZE][BLOCK_SIZE];

    if (row < rows && col < cols) 
    {
        
        s_in1[ty][tx] = d_in1[idx];
        s_in2[ty][tx] = d_in2[idx];
        __syncthreads();

        d_out[idx] = s_in1[ty][tx] + s_in2[ty][tx];
    }
}

void normalMatrixAdd(float* in1, float* in2, float* out, int rows, int cols)
{
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            int idx = i * cols + j;
            out[idx] = in1[idx] + in2[idx];
        }
    }
}

void unitTestMatrixAdd(float* out1, float* out2, int rows, int cols)
{
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            int idx = i * cols + j;
            assert(out1[idx] == out2[idx]);
        }
    }
    std::cout << "ALL TESTS PASSED\n";
}

int main(int argc, char** argv)
{
    const int ROWS = 1000;
    const int COLS = 500;
    const int MATRIX_SIZE = ROWS * COLS;

    const int THREADS_PER_BLOCK_X = BLOCK_SIZE;
    const int THREADS_PER_BLOCK_Y = BLOCK_SIZE;
    const int BLOCKS_X = (COLS + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X;
    const int BLOCKS_Y = (ROWS + THREADS_PER_BLOCK_Y - 1) / THREADS_PER_BLOCK_Y;

    float h_in1[MATRIX_SIZE];
    float h_in2[MATRIX_SIZE];
    for (int i = 0; i < ROWS; i++) {
        for (int j = 0; j < COLS; j++) {
            int idx = i * COLS + j;
            h_in1[idx] = float(idx);
            h_in2[idx] = float(idx * 2);
        }
    }

    float h_out[MATRIX_SIZE];
    float h_out_C[MATRIX_SIZE];

    TimePoint timer = Clock::now();
    normalMatrixAdd(h_in1, h_in2, h_out_C, ROWS, COLS);
    std::chrono::duration<double> sec = (Clock::now() - timer);
    double time = 1e6 * sec.count();
    std::cout << "Normal program takes " << time << " microseconds to complete." << std::endl;

    float* d_in1;
    float* d_in2;
    float* d_out;
    cudaMalloc((void**)&d_in1, MATRIX_SIZE * sizeof(float));
    cudaMalloc((void**)&d_in2, MATRIX_SIZE * sizeof(float));
    cudaMalloc((void**)&d_out, MATRIX_SIZE * sizeof(float));

    cudaMemcpy(d_in1, h_in1, MATRIX_SIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_in2, h_in2, MATRIX_SIZE * sizeof(float), cudaMemcpyHostToDevice);

    dim3 blockSize(THREADS_PER_BLOCK_X, THREADS_PER_BLOCK_Y);
    dim3 gridSize(BLOCKS_X, BLOCKS_Y);

    TimePoint timer1 = Clock::now();
    matrixAdd<<<gridSize, blockSize>>>(d_out, d_in1, d_in2, ROWS, COLS);
    std::chrono::duration<double> secs = (Clock::now() - timer1);
    double duration = 1e6 * secs.count();
    std::cout << "CUDA program takes " << duration << " microseconds to complete." << std::endl;

    cudaMemcpy(h_out, d_out, MATRIX_SIZE * sizeof(float), cudaMemcpyDeviceToHost);

    unitTestMatrixAdd(h_out, h_out_C, ROWS, COLS);

    cudaFree(d_in1);
    cudaFree(d_in2);
    cudaFree(d_out);

    return 0;
}


Normal program takes 2604.87 microseconds to complete.
CUDA program takes 25.967 microseconds to complete.
ALL TESTS PASSED



In [41]:
%%cu

#include <iostream>
#include <chrono>
#include <assert.h>

typedef std::chrono::high_resolution_clock Clock;
typedef std::chrono::time_point<Clock> TimePoint;

__global__ void matrixAdd(float* d_out, float* d_in1, float* d_in2, int rows, int cols)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int idx = row * cols + col;
    if (row < rows && col < cols) {
        d_out[idx] = d_in1[idx] + d_in2[idx];
    }
}

void normalMatrixAdd(float* in1, float* in2, float* out, int rows, int cols)
{
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            int idx = i * cols + j;
            out[idx] = in1[idx] + in2[idx];
        }
    }
}

void unitTestMatrixAdd(float* out1, float* out2, int rows, int cols)
{
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            int idx = i * cols + j;
            assert(out1[idx] == out2[idx]);
        }
    }
    std::cout << "ALL TESTS PASSED\n";
}

int main(int argc, char** argv)
{
    const int ROWS = 500;
    const int COLS = 500;
    const int MATRIX_SIZE = ROWS * COLS;

    const int THREADS_PER_BLOCK_X = 32;
    const int THREADS_PER_BLOCK_Y = 32;
    const int BLOCKS_X = (COLS + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X;
    const int BLOCKS_Y = (ROWS + THREADS_PER_BLOCK_Y - 1) / THREADS_PER_BLOCK_Y;

    float h_in1[MATRIX_SIZE];
    float h_in2[MATRIX_SIZE];
 
    for (int i = 0; i < ROWS; i++) {
        for (int j = 0; j < COLS; j++) {
            int idx = i * COLS + j;
            h_in1[idx] = float(idx);
            h_in2[idx] = float(idx * 2);
        }
    }

    float h_out[MATRIX_SIZE];
    float h_out_C[MATRIX_SIZE];

    TimePoint timer = Clock::now();
    normalMatrixAdd(h_in1, h_in2, h_out_C, ROWS, COLS);
    std::chrono::duration<double> sec = (Clock::now() - timer);
    double time = 1e6 * sec.count();
    std::cout << "Normal program takes " << time << " microseconds to complete." << std::endl;

    float* d_in1;
    float* d_in2;
    float* d_out;
    cudaMalloc((void**)&d_in1, MATRIX_SIZE * sizeof(float));
    cudaMalloc((void**)&d_in2, MATRIX_SIZE * sizeof(float));
    cudaMalloc((void**)&d_out, MATRIX_SIZE * sizeof(float));

    cudaMemcpy(d_in1, h_in1, MATRIX_SIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_in2, h_in2, MATRIX_SIZE * sizeof(float), cudaMemcpyHostToDevice);

    dim3 blockSize(THREADS_PER_BLOCK_X, THREADS_PER_BLOCK_Y);
 
    dim3 gridSize(BLOCKS_X, BLOCKS_Y);

    TimePoint timer1 = Clock::now();
 
    matrixAdd<<<gridSize, blockSize>>>(d_out, d_in1, d_in2, ROWS, COLS);
 
    std::chrono::duration<double> secs = (Clock::now() - timer1);
    double duration = 1e6 * secs.count();
    std::cout << "CUDA program takes " << duration << " microseconds to complete." << std::endl;

    cudaMemcpy(h_out, d_out, MATRIX_SIZE * sizeof(float), cudaMemcpyDeviceToHost);

    unitTestMatrixAdd(h_out, h_out_C, ROWS, COLS);

    cudaFree(d_in1);
    cudaFree(d_in2);
    cudaFree(d_out);

    return 0;
}


Normal program takes 885.112 microseconds to complete.
CUDA program takes 27.408 microseconds to complete.
ALL TESTS PASSED



In [43]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define N 2

__global__ void MatAdd(int A[][N], int B[][N], int C[][N]){
           int i = threadIdx.x;
           int j = threadIdx.y;

           C[i][j] = A[i][j] + B[i][j];
       }


int main(){

int A[N][N] = {{1,2},{3,4}};
int B[N][N] = {{5,6},{7,8}};
int C[N][N] = {{0,0},{0,0}};    

int (*pA)[N], (*pB)[N], (*pC)[N];

cudaMalloc((void**)&pA, (N*N)*sizeof(int));
cudaMalloc((void**)&pB, (N*N)*sizeof(int));
cudaMalloc((void**)&pC, (N*N)*sizeof(int));

cudaMemcpy(pA, A, (N*N)*sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(pB, B, (N*N)*sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(pC, C, (N*N)*sizeof(int), cudaMemcpyHostToDevice);

int numBlocks = 1;
dim3 threadsPerBlock(N,N);
MatAdd<<<numBlocks,threadsPerBlock>>>(pA,pB,pC);

cudaMemcpy(C, pC, (N*N)*sizeof(int), cudaMemcpyDeviceToHost);

int i, j; printf("C = \n");
for(i=0;i<N;i++){
    for(j=0;j<N;j++){
        printf("%d ", C[i][j]);
    }
    printf("\n");
}

cudaFree(pA); 
cudaFree(pB); 
cudaFree(pC);

printf("\n");

return 0;
}

C = 
6 8 
10 12 




In [86]:
%%cu
#include <iostream>
#include <cstdlib>
#include <cassert>

#define ROWS 32
#define COLS 32
#define THREADS_PER_BLOCK 1024

__global__ void MatAdd(int A[][COLS], int B[][COLS], int C[][COLS]){
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < ROWS && j < COLS) {
        C[i][j] = A[i][j] + B[i][j];
    }
}

void normalMatrixAdd(int A[][COLS], int B[][COLS], int C[][COLS]){
    for (int i = 0; i < ROWS; i++){
        for (int j = 0; j < COLS; j++){
            C[i][j] = A[i][j] + B[i][j];
        }
    }
}

void unitTestMatrixAdd(int C[][COLS], int C_normal[][COLS]){
    for (int i = 0; i < ROWS; i++){
        for (int j = 0; j < COLS; j++){
            assert(C[i][j] == C_normal[i][j]);
        }
    }
    std::cout << "ALL TESTS PASSED" << std::endl;
}

int main(){
    int h_in1[ROWS][COLS];
    int h_in2[ROWS][COLS];
    int h_out_C[ROWS][COLS];
    int h_out[ROWS][COLS];


    for (int i = 0; i < ROWS; i++){
        for (int j = 0; j < COLS; j++){
            h_in1[i][j] = i + j;
            h_in2[i][j] = i - j;
        }
    }

    normalMatrixAdd(h_in1,h_in2,h_out_C);

    int (*pA)[COLS], (*pB)[COLS], (*pC)[COLS];

    cudaMalloc((void**)&pA, (ROWS*COLS)*sizeof(int));
    cudaMalloc((void**)&pB, (ROWS*COLS)*sizeof(int));
    cudaMalloc((void**)&pC, (ROWS*COLS)*sizeof(int));

    cudaMemcpy(pA, h_in1, (ROWS*COLS)*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(pB, h_in2, (ROWS*COLS)*sizeof(int), cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(THREADS_PER_BLOCK);
    dim3 numBlocks((COLS + threadsPerBlock.x - 1) / threadsPerBlock.x, (ROWS + threadsPerBlock.y - 1) / threadsPerBlock.y);

    MatAdd<<<numBlocks, threadsPerBlock>>>(pA, pB, pC);

    cudaMemcpy(h_out, pC, (ROWS*COLS)*sizeof(int), cudaMemcpyDeviceToHost);

    unitTestMatrixAdd(h_out, h_out_C);

    cudaFree(pA);
    cudaFree(pB);
    cudaFree(pC);

    std::cout << std::endl;

    return 0;
}


ALL TESTS PASSED




# **FINAL** - Matrix Addition of 2d Arrays

In [89]:
%%cu
#include <iostream>
#include <cstdlib>
#include <assert.h>
#include <chrono>

#define ROWS 1000
#define COLS 100



typedef std::chrono::high_resolution_clock Clock;
typedef std::chrono::time_point<Clock> TimePoint;

__global__ void MatAdd(int A[][COLS], int B[][COLS], int C[][COLS]){
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < ROWS && j < COLS) {
        C[i][j] = A[i][j] + B[i][j];
    }
}

void normalMatrixAdd(int A[][COLS], int B[][COLS], int C[][COLS]){
    for (int i = 0; i < ROWS; i++){
        for (int j = 0; j < COLS; j++){
            C[i][j] = A[i][j] + B[i][j];
        }
    }
}

void unitTestMatrixAdd(int C[][COLS], int C_normal[][COLS]){
    for (int i = 0; i < ROWS; i++){
        for (int j = 0; j < COLS; j++){
            assert(C[i][j] == C_normal[i][j]);
        }
    }
    std::cout << "ALL TESTS PASSED" << std::endl;
}

int main(){
    int h_in1[ROWS][COLS];
    int h_in2[ROWS][COLS];
    int h_out_C[ROWS][COLS];
    int h_out[ROWS][COLS];

    int THREADS_PER_BLOCK_X = 32;
    int THREADS_PER_BLOCK_Y = 32;

    int BLOCKS_X = (COLS + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X;
    int BLOCKS_Y = (ROWS + THREADS_PER_BLOCK_Y - 1) / THREADS_PER_BLOCK_Y;


    for (int i = 0; i < ROWS; i++){
        for (int j = 0; j < COLS; j++){
            h_in1[i][j] = i + j;
            h_in2[i][j] = i - j;
        }
    }

    TimePoint timer = Clock::now();

    normalMatrixAdd(h_in1,h_in2,h_out_C);

    std::chrono::duration<double> sec = (Clock::now() - timer);
    double time = 1e6 * sec.count();
    std::cout << "Normal program takes " << time << " microseconds to complete." << std::endl;



    int (*pA)[COLS], (*pB)[COLS], (*pC)[COLS];

    cudaMalloc((void**)&pA, (ROWS*COLS)*sizeof(int));
    cudaMalloc((void**)&pB, (ROWS*COLS)*sizeof(int));
    cudaMalloc((void**)&pC, (ROWS*COLS)*sizeof(int));

    cudaMemcpy(pA, h_in1, (ROWS*COLS)*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(pB, h_in2, (ROWS*COLS)*sizeof(int), cudaMemcpyHostToDevice);

    
    dim3 blockSize(THREADS_PER_BLOCK_X, THREADS_PER_BLOCK_Y);

    dim3 numBlocks((ROWS + blockSize.x - 1) / blockSize.x, (COLS + blockSize.y - 1) / blockSize.y);

    TimePoint timer1 = Clock::now();

    MatAdd<<<numBlocks, blockSize>>>(pA, pB, pC);
    

    std::chrono::duration<double> secs = (Clock::now() - timer1);
    double duration = 1e6 * secs.count();
    std::cout << "CUDA program takes " << duration << " microseconds to complete." << std::endl;

    cudaMemcpy(h_out, pC, (ROWS*COLS)*sizeof(int), cudaMemcpyDeviceToHost);

    unitTestMatrixAdd(h_out, h_out_C);

    cudaFree(pA);
    cudaFree(pB);
    cudaFree(pC);

    std::cout << std::endl;

    return 0;
}


//FINAL

Normal program takes 411.102 microseconds to complete.
CUDA program takes 18.202 microseconds to complete.
ALL TESTS PASSED




In [98]:
%%cu
#include <iostream>

__global__ void staticReverse(int *d, int n)
{
  __shared__ int s[64];
  int t = threadIdx.x;
  int tr = n-t-1;
  s[t] = d[t];
  __syncthreads();
  d[t] = s[tr];
}



int main(void)
{
  const int n = 10;
  int a[n], r[n], d[n];

  for (int i = 0; i < n; i++) {
    a[i] = i;
    r[i] = n-i-1;
    d[i] = 0;
  }

  int *d_d;
  cudaMalloc(&d_d, n * sizeof(int)); 

  // Print input array
  std::cout << "Input Array: ";
  for (int i = 0; i < n; i++) {
    std::cout << a[i] << " ";
  }
  std::cout << std::endl;

  // run version with static shared memory
  cudaMemcpy(d_d, a, n*sizeof(int), cudaMemcpyHostToDevice);
  staticReverse<<<1,n>>>(d_d, n);
  cudaMemcpy(d, d_d, n*sizeof(int), cudaMemcpyDeviceToHost);
  
  // Print output array
  std::cout << "Output Array (Static): ";
  for (int i = 0; i < n; i++) {
    std::cout << d[i] << " ";
  }
  std::cout << std::endl;


  cudaFree(d_d);

  return 0;
}


Input Array: 0 1 2 3 4 5 6 7 8 9 
Output Array (Static): 9 8 7 6 5 4 3 2 1 0 





---


In this kernel, t and tr are the two indices representing the original and reverse order, respectively. Threads copy the data from global memory to shared memory with the statement s[t] = d[t], and the reversal is done two lines later with the statement d[t] = s[tr]. But before executing this final line in which each thread accesses data in shared memory that was written by another thread, remember that we need to make sure all threads have completed the loads to shared memory, by calling __syncthreads().

---

[Reference](https://developer.nvidia.com/blog/using-shared-memory-cuda-cc/)


# Same code using Shared Memory!

In [108]:
%%cu
#include <iostream>
#include <cstdlib>
#include <assert.h>
#include <chrono>

#define ROWS 1000
#define COLS 1000

typedef std::chrono::high_resolution_clock Clock;
typedef std::chrono::time_point<Clock> TimePoint;

__global__ void MatAdd(int A[][COLS], int B[][COLS], int C[][COLS])
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    // Define shared memory
    __shared__ int sharedA[32][32];
    __shared__ int sharedB[32][32];

    if (i < ROWS && j < COLS)
    {
        // Load data into shared memory
        sharedA[threadIdx.x][threadIdx.y] = A[i][j];
        sharedB[threadIdx.x][threadIdx.y] = B[i][j];

        // Synchronize threads to ensure all data is loaded into shared memory
        __syncthreads();

        // Perform element-wise addition using shared memory
        C[i][j] = sharedA[threadIdx.x][threadIdx.y] + sharedB[threadIdx.x][threadIdx.y];
    }
}

void normalMatrixAdd(int A[][COLS], int B[][COLS], int C[][COLS])
{
    for (int i = 0; i < ROWS; i++)
    {
        for (int j = 0; j < COLS; j++)
        {
            C[i][j] = A[i][j] + B[i][j];
        }
    }
}

void unitTestMatrixAdd(int C[][COLS], int C_normal[][COLS])
{
    for (int i = 0; i < ROWS; i++)
    {
        for (int j = 0; j < COLS; j++)
        {
            assert(C[i][j] == C_normal[i][j]);
        }
    }
    std::cout << "ALL TESTS PASSED" << std::endl;
}

int main()
{
    int h_in1[ROWS][COLS];
    int h_in2[ROWS][COLS];
    int h_out_C[ROWS][COLS];
    int h_out[ROWS][COLS];

    int THREADS_PER_BLOCK_X = 32;
    int THREADS_PER_BLOCK_Y = 32;

    for (int i = 0; i < ROWS; i++)
    {
        for (int j = 0; j < COLS; j++)
        {
            h_in1[i][j] = i + j;
            h_in2[i][j] = i - j;
        }
    }

    TimePoint timer = Clock::now();

    normalMatrixAdd(h_in1, h_in2, h_out_C);

    std::chrono::duration<double> sec = (Clock::now() - timer);
    double time = 1e6 * sec.count();
    std::cout << "Normal program takes " << time << " microseconds to complete." << std::endl;

    int (*pA)[COLS], (*pB)[COLS], (*pC)[COLS];

    cudaMalloc((void **)&pA, (ROWS * COLS) * sizeof(int));
    cudaMalloc((void **)&pB, (ROWS * COLS) * sizeof(int));
    cudaMalloc((void **)&pC, (ROWS * COLS) * sizeof(int));

    cudaMemcpy(pA, h_in1, (ROWS * COLS) * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(pB, h_in2, (ROWS * COLS) * sizeof(int), cudaMemcpyHostToDevice);

    dim3 blockSize(THREADS_PER_BLOCK_X, THREADS_PER_BLOCK_Y);
    dim3 numBlocks((ROWS + blockSize.x - 1) / blockSize.x, (COLS + blockSize.y - 1) / blockSize.y);

    TimePoint timer1 = Clock::now();

    MatAdd<<<numBlocks, blockSize>>>(pA, pB, pC);

    std::chrono::duration<double> secs = (Clock::now() - timer1);
    double duration = 1e6 * secs.count();
    std::cout << "CUDA program takes " << duration << " microseconds to complete." << std::endl;

    cudaMemcpy(h_out, pC, (ROWS * COLS) * sizeof(int), cudaMemcpyDeviceToHost);

    unitTestMatrixAdd(h_out, h_out_C);

    cudaFree(pA);
    cudaFree(pB);
    cudaFree(pC);

    std::cout << std::endl;

    return 0;
}
